In [12]:
import camelot

# Парсим дивиденды из PDF, которую нам предоставляет тинькофф

tables = camelot.read_pdf('resources/divs.pdf', pages = "1-end")

In [20]:
# Объединяем в одну таблицу

import pandas as pd

# concat data frames
df = pd.concat([x.df for x in tables])

# remove redundant rows and transform it to table header
df.columns = df.iloc[0]

df = df.drop(0)

In [86]:
# Запускаем хром логинимся, переходим на форму, где мы должны указать доходы из-за рубежа

from selenium import webdriver

driver = webdriver.Chrome()

In [92]:
# Собственно, когда все готово - запускаем эту яйечку

countries_dict = {
    'США': '840 - СОЕДИНЕННЫЕ ШТАТЫ ',
    'Кипр': '196 - КИПР ',
    'Великобритания': '826 - СОЕДИНЕННОЕ КОРОЛЕВСТВО ',
    'Нидерланды': '528 - НИДЕРЛАНДЫ ',
    'Швейцарская Конфедерация': '756 - ШВЕЙЦАРИЯ ',
    'Ирландия': '372 - ИРЛАНДИЯ ',
    'КАЙМАН': '136 - ОСТРОВА КАЙМАН ',
}

income_type = '1010 - Дивиденды'
i = 0

add_button = driver.find_element_by_class_name('src-modules-Taps-components-NDFL3-private-forms-IncomesForm-IncomesOutsideRFComponent-IncomeSources-IncomeSources-module__addButton')
for row in df.iterrows():
    name = row[1].values[3].replace(' ORD SHS', '')
    country = row[1].values[5].replace('\n', ' ')
    value_without_tax = row[1].values[9]
    tax_paid = row[1].values[10]
    date_received = row[1].values[1]
    date_paid = row[1].values[1]

    add_button.click()
    driver.find_elements_by_class_name('src-modules-Taps-components-NDFL3-private-forms-IncomesForm-IncomesOutsideRFComponent-IncomeSources-IncomeSources-module__incomeSourceSpoiler')[-1].click()
    # name
    driver.find_element_by_name(f'Ndfl3Package.payload.sheetB.sources[{i}].incomeSourceName').send_keys(name)

    # country
    driver.find_element_by_id(f'Ndfl3Package.payload.sheetB.sources[{i}].oksm').click()
    driver.find_element_by_css_selector(f"div[aria-label='{countries_dict[country]}']").click()

    # revenue type
    driver.find_element_by_id(f'Ndfl3Package.payload.sheetB.sources[{i}].incomeTypeCode').click()
    driver.find_element_by_css_selector(f"div[aria-label='{income_type}']").click()

    # deduct_type
    deduct_id = f'Ndfl3Package.payload.sheetB.sources[{i}].taxDeductionCode'
    deduct_type = 'Не предоставлять вычет'
    driver.find_element_by_id(deduct_id).click()
    driver.find_element_by_css_selector(f"div[aria-label='{deduct_type}']").click()

    # revenue
    revenue_id = f'Ndfl3Package.payload.sheetB.sources[{i}].incomeAmountCurrency'
    driver.find_element_by_id(revenue_id).send_keys(value_without_tax.replace(',', '.'))

    # dates
    income_date_id = f'Ndfl3Package.payload.sheetB.sources[{i}].incomeDate'
    tax_date_id = f'Ndfl3Package.payload.sheetB.sources[{i}].taxPaymentDate'

    driver.find_element_by_id(income_date_id).find_element_by_tag_name('input').send_keys(date_received)
    driver.find_element_by_id(tax_date_id).find_element_by_tag_name('input').send_keys(date_paid)

    # currency
    currency = '840 - Доллар США'
    currency_id = f'Ndfl3Package.payload.sheetB.sources[{i}].currencyCode'
    driver.find_element_by_id(currency_id).click()
    driver.find_element_by_css_selector(f"div[aria-label='{currency}']").click()

    # get currency quote automatically
    driver.find_elements_by_class_name('form-checkbox')[-1].click()

    # already paid
    driver.find_element_by_id(f'Ndfl3Package.payload.sheetB.sources[{i}].paymentAmountCurrency').send_keys(tax_paid.replace(',', '.'))

    i += 1

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=89.0.4389.114)
